In [ ]:
!pip install yfinance numpy pandas scikit-learn tensorflow keras-tuner scikit-optimize matplotlib

In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from sklearn.metrics import r2_score, explained_variance_score

import matplotlib.pyplot as plt

In [3]:
# List of ticker symbols
tickers = [
    "MQG.AX","ALL.AX", "ANZ.AX", "BHP.AX", "CBA.AX", "CSL.AX", "FMG.AX", 
    "GMG.AX", "NAB.AX", "REA.AX", "RIO.AX", "TCL.AX", 
    "TLS.AX", "WBC.AX", "WES.AX", "WOW.AX", "WDS.AX", "XRO.AX"
]

# Calculate the date 10 years ago from today, or use a fixed start date
start_date = (datetime.now() - timedelta(days=365*10)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')  # Today's date for the end date

# Fetch the stock data and store in a list of DataFrames
stock_data = []
for ticker in tickers:
    df = yf.download(ticker, start=start_date, end=end_date, interval='1wk')
    stock_data.append(df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [4]:
# define target
for i in range(0,len(stock_data)):
    stock_data[i]['Target'] = stock_data[i]['Close'].shift(-1) # -1 used to used next day's price as target

In [5]:
# add indicators
for i in stock_data:
    data = i
    # SMA-10
    data['SMA_10'] = data['Close'].rolling(window=10).mean().shift(1)
    # SMA 25
    data['SMA_25'] = data['Close'].rolling(window=25).mean().shift(1)

    # Momentum (Close price difference shifted by 1 to use previous day's data)
    data['Momentum'] = data['Close'].diff().shift(1)

    # Stochastic %K and %D using data up to the previous day
    low_min = data['Low'].rolling(window=14).min().shift(1)
    high_max = data['High'].rolling(window=14).max().shift(1)
    data['Stochastic_%K'] = ((data['Close'].shift(1) - low_min) / (high_max - low_min)) * 100
    data['Stochastic_%D'] = data['Stochastic_%K'].rolling(window=3).mean()

    # RSI calculated from previous days
    delta = data['Close'].diff(1).shift(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))

    # MACD calculation shifted to include only past data
    exp1 = data['Close'].ewm(span=12, adjust=False).mean().shift(1)
    exp2 = data['Close'].ewm(span=26, adjust=False).mean().shift(1)
    macd = exp1 - exp2
    signal = macd.ewm(span=9, adjust=False).mean()
    data['MACD'] = macd
    data['MACD_Signal'] = signal

    # list of columns
    columns = ['Open', 'High', 'Low', 'Close', 'Volume',
               'Target', 'SMA_10', 'SMA_25', 'Momentum', 'Stochastic_%K', 'Stochastic_%D', 'RSI',
               'MACD', 'MACD_Signal']

    # reorder columns to put 'Target' in the end
    columns.remove('Target')
    columns.append('Target')

    data= data[columns]
        


In [6]:
for i in range(len(stock_data)):
    # Drop all rows that contain any NaN values
    stock_data[i].dropna(inplace=True)


In [7]:
# check for null values if needed
# for i, df in enumerate(stock_data):
#     if df.isna().any().any():  # Check if any value in the DataFrame is NaN
#         print(f"DataFrame at index {i} still contains NaN values.")
#     else:
#         print(f"DataFrame at index {i} is clean of NaN values.")

In [8]:

scaled_features = []  # List to store feature arrays for each DataFrame
scaled_targets = []  # List to store target arrays for each DataFrame

# Instantiate scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Loop through each DataFrame
for df in stock_data:
    if not df.empty and df.shape[1] > 1:  # Ensure DataFrame has more than one column
        scaled_data = scaler.fit_transform(df.values)  # Scale the data

        # Split scaled data into features and target
        features = scaled_data[:, :-1]
        target = scaled_data[:, -1]

        # Store the scaled features and targets in their respective lists
        scaled_features.append(features)
        scaled_targets.append(target)

In [9]:
# Assuming scaled_features and scaled_targets are your lists of feature and target arrays from multiple datasets
X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []


# Split each dataset
for features, target in zip(scaled_features, scaled_targets):
    split_idx = int(len(features) * 0.7)  # 70% for training
    X_train_list.append(features[:split_idx])
    X_test_list.append(features[split_idx:])
    y_train_list.append(target[:split_idx])
    y_test_list.append(target[split_idx:])
    
# for i, (feat, targ) in enumerate(zip(scaled_features, scaled_targets)):
#     print(f"Features shape for DataFrame {i}: {feat.shape}")
#     print(f"Target shape for DataFrame {i}: {targ.shape}")

In [ ]:
model = Sequential([
    Dense(128, input_dim=X_train_list[0].shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),  # Additional layer
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')

for X_train, y_train in zip(X_train_list, y_train_list):
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

C:\Users\Kirill\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1430  
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 0.0300
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 0.0173
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - loss: 0.0107
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - loss: 0.0099
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 0.0066
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - loss: 0.0052
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 0.0039
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 0.0028
Epoch 10/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - loss: 0.0020
Epoch 11/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0014     
Epoch 12/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0011 
Epoch 13/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.6902e-04 
Epoch 14/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step - loss: 8.4066e-04
Epoch 15/100
11/11 ━━━━━━━━━━━━━━

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 1.9472e-04
Epoch 55/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - loss: 2.3139e-04
Epoch 56/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 2.6620e-04
Epoch 57/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 2.7775e-04
Epoch 58/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 2.8962e-04
Epoch 59/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - loss: 3.1832e-04
Epoch 60/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - loss: 1.9993e-04
Epoch 61/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 897us/step - loss: 2.0240e-04
Epoch 62/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: 1.7758e-04
Epoch 63/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - loss: 2.1160e-04
Epoch 64/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 1.7535e-04
Epoch 65/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 3.4382e-04
Epoch 66/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 1.8144e-04
Epoch 67/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: 1.

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - loss: 2.2440e-04
Epoch 31/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 864us/step - loss: 2.3520e-04
Epoch 32/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 2.1306e-04
Epoch 33/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 2.8322e-04
Epoch 34/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 2.7414e-04
Epoch 35/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 2.5455e-04
Epoch 36/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - loss: 2.7925e-04
Epoch 37/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: 2.1630e-04
Epoch 38/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - loss: 2.7452e-04
Epoch 39/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - loss: 2.4282e-04
Epoch 40/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - loss: 2.4034e-04
Epoch 41/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - loss: 2.1631e-04
Epoch 42/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - loss: 2.7171e-04
Epoch 43/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: 3.

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - loss: 4.3836e-04
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 4.3740e-04
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: 6.4815e-04
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 4.2893e-04
Epoch 10/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: 3.8517e-04
Epoch 11/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 4.7784e-04
Epoch 12/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 3.9185e-04
Epoch 13/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 3.6560e-04
Epoch 14/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - loss: 4.2030e-04
Epoch 15/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 3.3016e-04
Epoch 16/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: 3.2285e-04
Epoch 17/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 3.7525e-04
Epoch 18/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 3.0563e-04
Epoch 19/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step - loss: 3.103

...

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - loss: 9.0572e-05
Epoch 47/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 9.7461e-05
Epoch 48/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 1.0059e-04
Epoch 49/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 650us/step - loss: 1.4815e-04
Epoch 50/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - loss: 1.2884e-04
Epoch 51/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: 9.8917e-05
Epoch 52/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 7.6366e-05
Epoch 53/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step - loss: 8.8006e-05
Epoch 54/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 8.4975e-05
Epoch 55/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step - loss: 8.0701e-05
Epoch 56/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 8.8579e-05
Epoch 57/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 7.2238e-05
Epoch 58/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.6231e-05 
Epoch 59/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step - loss: 7.0

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 650us/step - loss: 3.8609e-04
Epoch 23/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 2.9292e-04
Epoch 24/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 3.0238e-04
Epoch 25/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: 3.0002e-04
Epoch 26/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 650us/step - loss: 3.0952e-04
Epoch 27/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 669us/step - loss: 3.4854e-04
Epoch 28/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - loss: 2.8545e-04
Epoch 29/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: 2.9097e-04
Epoch 30/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - loss: 3.2146e-04
Epoch 31/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 2.8054e-04
Epoch 32/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - loss: 2.8893e-04
Epoch 33/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 760us/step - loss: 2.7956e-04
Epoch 34/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 650us/step - loss: 2.5375e-04
Epoch 35/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 3.

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 1.2916e-04
Epoch 99/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 1.7930e-04
Epoch 100/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 1.2860e-04
Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - loss: 0.0033
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - loss: 8.2409e-04
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 6.8409e-04
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - loss: 5.7740e-04
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - loss: 4.1371e-04
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - loss: 3.7062e-04
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step - loss: 3.8246e-04
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 4.7889e-04
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 4.1566e-04
Epoch 10/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - loss: 2.8419e-04
Epoch 11/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - loss: 2.3324e-04
Epo

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3002e-04 
Epoch 75/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - loss: 1.3075e-04
Epoch 76/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - loss: 1.4990e-04
Epoch 77/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step - loss: 1.5510e-04
Epoch 78/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - loss: 1.2077e-04
Epoch 79/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - loss: 1.3808e-04
Epoch 80/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - loss: 2.3786e-04
Epoch 81/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - loss: 4.4815e-04
Epoch 82/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 890us/step - loss: 2.5217e-04
Epoch 83/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step - loss: 1.4934e-04
Epoch 84/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 857us/step - loss: 1.7875e-04
Epoch 85/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step - loss: 1.4932e-04
Epoch 86/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - loss: 1.5235e-04
Epoch 87/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - loss: 2.7

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - loss: 1.0400e-04
Epoch 51/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - loss: 6.8599e-05
Epoch 52/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step - loss: 5.9134e-05
Epoch 53/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 673us/step - loss: 5.1481e-05
Epoch 54/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 6.9998e-05
Epoch 55/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - loss: 7.7019e-05
Epoch 56/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 6.7022e-05
Epoch 57/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - loss: 6.7625e-05
Epoch 58/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 6.6308e-05
Epoch 59/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 5.4662e-05
Epoch 60/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - loss: 5.9009e-05
Epoch 61/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 7.2050e-05
Epoch 62/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - loss: 5.9351e-05
Epoch 63/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - loss: 5.

In [ ]:
predictions = model.predict(X_test_list[0]) # MQG.AX stock

# evaluation
y_pred = predictions
y_test = y_test_list[0] # MQG.AX stock



explained_variance = explained_variance_score(y_test, predictions)
print("Explained Variance Score:", explained_variance)


# Mean Squared Error (MSE)
mse = MeanSquaredError()
mse.update_state(y_test, y_pred)
mse_result = mse.result().numpy()
print("Mean Squared Error (MSE):", mse_result)

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse_result)
print("Root Mean Squared Error (RMSE):", rmse)

# Mean Absolute Error (MAE)
mae = MeanAbsoluteError()
mae.update_state(y_test, y_pred)
mae_result = mae.result().numpy()
print("Mean Absolute Error (MAE):", mae_result)

# r^2
r2 = r2_score(y_test, y_pred)
print("R^2 (Coefficient of Determination):", r2)

# Calculate Mean Bias Deviation (MBD)
def mean_bias_deviation(y_true, y_pred):
    differences = y_true - y_pred
    mbd = np.mean(differences)
    return mbd

# Calculate MBD for the test set
mbd = mean_bias_deviation(y_test, y_pred)
print("Mean Bias Deviation:", mbd)

In [ ]:
# plot results for macquarie group stock
y_test = np.array(y_test)
predictions = np.array(predictions)

plt.figure(figsize=(10, 5))
plt.plot(range(len(y_test)), y_test, label='Actual Values', color='blue', marker='o')
plt.plot(range(len(predictions)), predictions, label='Predicted Values', color='red', linestyle='--', marker='x')
plt.title('Actual vs. Predicted Values. ANN trained on all datasets')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()